<a href="https://colab.research.google.com/github/MeenTers/Project/blob/main/Project_SI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
from torch import nn, load, utils
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms, models
from torchvision.models.vgg import model_urls
from os import path, listdir
model_urls['vgg19'] = model_urls['vgg19'].replace('https://', 'http://')

In [3]:
from scipy.spatial import distance_matrix
from scipy.spatial import distance

# Prepearing the model

In [4]:
class VGG:
	def __init__(self):
		model = models.vgg19(pretrained=True, progress=True)
		model.classifier = nn.Sequential(*list(model.classifier.children())[:3])
		self.model = model.cuda().eval()

	def __call__(self, x):
		return self.model(x)

In [5]:
vgg = VGG()

# Features Extraction

In [6]:
def get_features(model, loader):
    features = []
    with torch.no_grad():
        for batch, _ in tqdm(loader):
            if torch.cuda.is_available():
                batch = batch.cuda()
            b_features = model(batch).detach().cpu().numpy()
            for f in b_features:
                features.append(f)

    return features

# Loading the data

In [7]:
def get_dataset(images_path):
  transform = transforms.Compose([
    transforms.Resize(size=32),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  dataset = datasets.ImageFolder(images_path, transform=transform)
  loader = utils.data.DataLoader(dataset, batch_size=100, shuffle=False, num_workers=1, pin_memory=True)
  return dataset, loader

# Test multi-view image

In [8]:
car_dataset , car_loader = get_dataset(r'/content/drive/MyDrive/Project_si/car_model/')

In [9]:
damage_dataset , damage_loader = get_dataset(r'/content/drive/MyDrive/Project_si/model_damage/')

In [10]:
car_base = get_features(model=vgg,loader=car_loader)

100%|██████████| 1/1 [00:00<00:00,  5.26it/s]


In [11]:
len(car_base[0])

4096

In [12]:
car_base[0]

array([0.      , 0.      , 3.960284, ..., 0.      , 0.      , 0.      ],
      dtype=float32)

In [13]:
car_damage = get_features(model=vgg,loader=damage_loader)

100%|██████████| 1/1 [00:00<00:00,  5.92it/s]


In [14]:
len(car_damage[0])

4096

# Distance

In [15]:
car_image = '/content/drive/MyDrive/Project_si/car_model/ID1'
damage_image = '/content/drive/MyDrive/Project_si/model_damage/ID1'

In [16]:
distance.euclidean(car_base[1], car_damage[1])

13.258048057556152

# compute distance


# ID1

In [30]:
base_car_dataset, base_car_loader = get_dataset('/content/drive/MyDrive/Project_si/Test_Dist/Base_car/ID1') 
view_car_dataset, view_car_loader = get_dataset('/content/drive/MyDrive/Project_si/Test_Dist/different_view/ID1')

In [18]:
base_car = get_features(vgg,base_car_loader)
view_car = get_features(vgg,view_car_loader)

100%|██████████| 1/1 [00:04<00:00,  4.37s/it]


In [19]:
distance.euclidean(base_car[0], view_car[0])

39.94890594482422

In [20]:
distance.euclidean(base_car[0], view_car[1])

37.413814544677734

In [21]:
distance.euclidean(base_car[0], view_car[2])

44.71876525878906

In [22]:
distance.euclidean(base_car[0], view_car[3])

49.92780685424805

In [23]:
distance.euclidean(base_car[0], view_car[4])

42.918880462646484

In [24]:
distance.euclidean(base_car[0], view_car[5])

48.43491744995117

In [25]:
distance.euclidean(base_car[0], view_car[6])

43.1046142578125

# ID2

In [29]:
base_car1_dataset, base_car1_loader = get_dataset('/content/drive/MyDrive/Project_si/Test_Dist/Base_car/ID2') 
view_car1_dataset, view_car1_loader = get_dataset('/content/drive/MyDrive/Project_si/Test_Dist/different_view/ID2')

In [31]:
base_car1 = get_features(vgg,base_car1_loader)
view_car1 = get_features(vgg,view_car1_loader)

100%|██████████| 1/1 [00:02<00:00,  2.76s/it]


In [32]:
distance.euclidean(base_car1[0], view_car1[0])

9.628503799438477

In [33]:
distance.euclidean(base_car1[0], view_car1[1])

23.583209991455078

In [34]:
distance.euclidean(base_car1[0], view_car1[2])

30.316486358642578

In [35]:
distance.euclidean(base_car1[0], view_car1[3])

28.636341094970703